# Тестирование Spell Checker с использованием дополнительных признаков

В этом ноутбуке будет произведено тестирование модели, в которой в candidate scorer используются модели на основе множества признаков помимо BERT.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import sys
import os
import json
import pickle
import re
from string import punctuation
sys.path.append('..')

import dotenv
import numpy as np
import pandas as pd
from transformers import BertForMaskedLM, BertTokenizer, BertConfig

from deeppavlov.core.data.simple_vocab import SimpleVocabulary

import kenlm
from sacremoses import MosesTokenizer, MosesDetokenizer

from src.models.SpellChecker import *
from src.models.BertScorer.bert_scorer_correction import (
    BertScorerCorrection
)

from sklearn.svm import LinearSVC

from IPython.display import display
from tqdm.notebook import tqdm

[nltk_data] Downloading package punkt to /home/mrgeekman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [3]:
PROJECT_PATH = os.path.join(os.path.abspath(''), os.pardir)
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
MODEL_PATH = os.path.join(PROJECT_PATH, 'models')

## Инициализация

Начнем с того, что инициализируем все необходимые компоненты модели. Параллельно так же будет описана роль каждого компонента в системе.

In [4]:
raw_tokenizer = MosesTokenizer(lang='ru')
raw_detokenizer = MosesDetokenizer(lang='ru')
tokenizer = lambda x: raw_tokenizer.tokenize(x, escape=False)
detokenizer = lambda x: raw_detokenizer.detokenize(x)

In [5]:
vocab_path = os.path.join(DATA_PATH, 'external', 'russian_words', 
                          'russian_words_vocab.dict')
vocab = SimpleVocabulary(load_path=vocab_path, save_path=vocab_path)
handcode_table_path = os.path.join(DATA_PATH, 'processed', 'handcode_table', 
                                   'table.json')
with open(handcode_table_path, 'r') as inf:
    handcode_table = json.load(inf)
candidate_generator = CandidateGenerator(
    words=vocab.keys(), handcode_table=handcode_table, max_distance=1
)

2021-01-30 12:06:12.827 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/mrgeekman/Documents/MIPT/НИР/Repo/data/external/russian_words/russian_words_vocab.dict]


In [6]:
model_left_right = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'left_right_3_100.arpa.binary')
)
model_right_left = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'right_left_3_100.arpa.binary')
)
position_selector = KenlmPositionSelector(model_left_right, model_right_left)

In [7]:
BERT_PATH = os.path.join(MODEL_PATH, 'conversational_rubert')
config = BertConfig.from_json_file(
    os.path.join(BERT_PATH, 'bert_config.json')
)
model = BertForMaskedLM.from_pretrained(
    os.path.join(BERT_PATH, 'pytorch_model.bin'),
    config=config
)
bert_tokenizer = BertTokenizer(os.path.join(BERT_PATH, 'vocab.txt'))
bert_scorer_correction = BertScorerCorrection(model, bert_tokenizer)
agg_subtoken_func = np.mean
bert_scorer = BertScorer(
    bert_scorer_correction, agg_subtoken_func
)

with open(os.path.join(MODEL_PATH, 'scorer', 'svm.bin'), 'rb') as inf:
    svm_model = pickle.load(inf)

svm_scorer = SVMScorer(svm_model, bert_scorer=bert_scorer)
candidate_scorer = CandidateScorer(svm_scorer)

Some weights of the model checkpoint at /home/mrgeekman/Documents/MIPT/НИР/Repo/notebooks/../models/conversational_rubert/pytorch_model.bin were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at /home/mrgeekman/Documents/MIPT/НИР/Repo/notebooks/../models/conversational_rubert/pytorch_model.bin and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a d

In [8]:
stopping_criteria = MarginStoppingCriteria(np.log(2.5))

In [9]:
# максимальное количество итераций
max_it = 5

spellchecker = IterativeSpellChecker(
    candidate_generator,
    position_selector,
    candidate_scorer,
    stopping_criteria,
    tokenizer,
    detokenizer,
    num_selected_candidates=None,
    max_it=max_it
)

## Тестирование

### Валидация

In [10]:
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 'train_source.txt'), 
    'r'
) as inf:
    sentences = inf.readlines()
    
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 
                 'train_corrected.txt'), 
    'r'
) as inf:
    true_sentences = inf.readlines()

Запустим наш spell checker, подавая ему предложения батчами размера `batch_size`.

In [11]:
batch_size = 5
sentences_corrected = []
num_batches = int(np.ceil(len(sentences) // batch_size))

for i in tqdm(range(num_batches)):
    cur_sentences = sentences[i*batch_size:(i+1)*batch_size]
    sentences_corrected += spellchecker(cur_sentences)

Запишем результаты файл.

In [12]:
!mkdir -p ../data/processed/results_svm/

In [13]:
with open(os.path.join(DATA_PATH, 'processed', 'results_svm', 'validation.txt'), 'w') as ouf:
    ouf.writelines([sentence + '\n' for sentence in sentences_corrected])

Выполним скрит для измерения качества.

In [14]:
!python ../src/evaluation/spell_ru_eval/evaluate.py -d ../data/processed/results_svm/diffs_validation.txt ../data/external/spell_ru_eval/train_source.txt ../data/external/spell_ru_eval/train_corrected.txt ../data/processed/results_svm/validation.txt | tail -n 2

Precision=90.38 Recall=75.62 FMeasure=82.35
1306 1445 1727


**Ranking SVM**

* True Positive: $1306$
* Внесенных исправлений: $1445$ 
* Требуемых исправления: $1727$ 
* Precision: $90.38$
* Recall: $75.62$
* FMeasure: $82.35$

**CatBoost**

* True Positive: $1354$
* Внесенных исправлений: $1453$ 
* Требуемых исправления: $1727$ 
* Precision: $93.19$
* Recall: $78.40$
* FMeasure: $85.16$

Как видим, тут CatBoost дал прирост как по recall, так и по precision.

### Тест

In [15]:
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 'test_source.txt'), 
    'r'
) as inf:
    sentences = inf.readlines()
    
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 
                 'test_corrected.txt'), 
    'r'
) as inf:
    true_sentences = inf.readlines()

Запустим наш spell checker, подавая ему предложения батчами размера `batch_size`.

In [16]:
batch_size = 5
sentences_corrected = []
num_batches = int(np.ceil(len(sentences) // batch_size))

for i in tqdm(range(num_batches)):
    cur_sentences = sentences[i*batch_size:(i+1)*batch_size]
    sentences_corrected += spellchecker(cur_sentences)

Запишем результаты файл.

In [17]:
with open(os.path.join(DATA_PATH, 'processed', 'results_svm', 'test.txt'), 'w') as ouf:
    ouf.writelines([sentence + '\n' for sentence in sentences_corrected])

Выполним скрит для измерения качества.

In [18]:
!python ../src/evaluation/spell_ru_eval/evaluate.py -d ../data/processed/results_svm/diffs_test.txt ../data/external/spell_ru_eval/test_source.txt ../data/external/spell_ru_eval/test_corrected.txt ../data/processed/results_svm/test.txt | tail -n 2

Precision=87.18 Recall=70.33 FMeasure=77.86
1387 1591 1972


**Ranking SVM**

* True Positive: $1387$
* Внесенных исправлений: $1591$ 
* Требуемых исправления: $1972$ 
* Precision: $87.18$
* Recall: $70.33$
* FMeasure: $77.86$

**CatBoost**

* True Positive: $1350$
* Внесенных исправлений: $1556$ 
* Требуемых исправления: $1972$
* Precision: $86.76$
* Recall: $68.46$
* FMeasure: $76.53$

На этот раз показатили CatBoost оказались хуже. Возможно, имеет место переобучение или некоторое отличие тестового множества от валидационного.

## Выводы

1. Результаты удалось заметно улучшить и достичь SOTA.
2. CatBoost дал улучшение на валидации, но на тесте лучше оказалась модель на SVM.